In [66]:
import numpy as np
import os
import h5py


## Clean up Jens' data and save as .hdf5 file

In [67]:
jens_folder = 'test_data/jens'
jens_files = os.listdir(jens_folder)
xueshi_folder = 'test_data/xueshi'
xueshi_files = os.listdir(xueshi_folder)

In [68]:
sq_spec_files = [os.path.join(jens_folder,f) for f in sorted(jens_files) if 'SPEC' in f and '_S' in f]
asq_spec_files = [os.path.join(jens_folder,f) for f in sorted(jens_files) if 'SPEC' in f and '_A' in f]
elec_spec_file = os.path.join(jens_folder, 'SPEC_elec.csv')
vac_spec_file = os.path.join(jens_folder, 'SPEC_vac.csv')

sq_power_files = [os.path.join(jens_folder, f) for f in sorted(jens_files) if '5MHz' in f and '_S' in f]
asq_power_files = [os.path.join(jens_folder, f) for f in sorted(jens_files) if '5MHz' in f and '_A' in f]
elec_power_file = os.path.join(jens_folder, '5MHz_elec.csv')
vac_power_file = os.path.join(jens_folder, '5MHz_vac.csv')

In [69]:
#SPEC data
electronic = np.genfromtxt(elec_spec_file, delimiter=',', skip_header=45)
vacuum = np.genfromtxt(vac_spec_file, delimiter=',', skip_header=45)
squeezing = np.array([np.genfromtxt(f, delimiter=',', skip_header=45) for f in sq_spec_files])
antisqueezing = np.array([np.genfromtxt(f, delimiter=',', skip_header=45) for f in asq_spec_files])


#frequencies (x-axis in each dataset above, as seen from the metadata). 
freq = electronic[:,0] 
electronic = electronic[:,1]
vacuum = vacuum[:,1]
squeezing = squeezing[:,:,1]
antisqueezing = antisqueezing[:,:,1] 

meta_elec = np.genfromtxt(elec_spec_file, delimiter=',', skip_header=2, max_rows=42, dtype='S')
#Format meta data
meta_elec = {a[0]:a[1] for a in meta_elec}

#meta_elec

In [70]:
#5MHz data
electronic_5 = np.genfromtxt(elec_power_file, delimiter=',', skip_header=45)
vacuum_5 = np.genfromtxt(vac_power_file, delimiter=',', skip_header=45)
squeezing_5 = np.array([np.genfromtxt(f, delimiter=',', skip_header=45) for f in sq_power_files])
antisqueezing_5 = np.array([np.genfromtxt(f, delimiter=',', skip_header=45) for f in asq_power_files])

t = electronic_5[:,0]
power = np.array([float(f.split('_')[2]) for f in sq_power_files])
electronic_5 = electronic_5[:,1]
vacuum_5 = vacuum_5[:,1]
squeezing_5 = squeezing_5[:,:,1]
antisqueezing_5 = antisqueezing_5[:,:,1] 


#Extract powers 

meta_elec_5 = np.genfromtxt(elec_power_file, delimiter=',', skip_header=2, max_rows=42, dtype='S')
#Format metadata
meta_elec_5 = {a[0]:a[1] for a in meta_elec_5}


In [71]:
with h5py.File('squeezing_data.hdf5', 'w') as file:
    dset_spec = file.create_group('spectrum')
    dset_freq = dset_spec.create_dataset('frequencies', data=freq)
    dset_elec = dset_spec.create_dataset('electronic', data=electronic)
    dset_vac = dset_spec.create_dataset('vacuum', data=vacuum)
    dset_sqz = dset_spec.create_dataset('squeezing', data=squeezing)
    dset_asqz = dset_spec.create_dataset('antisqueezing', data=antisqueezing)

    dset_5 = file.create_group('5MHz')
    dset_time = dset_5.create_dataset('times', data = t)
    dset_power = dset_5.create_dataset('power', data=power)
    dset_elec_5 = dset_5.create_dataset('electronic', data = electronic_5)
    dset_vac_5 = dset_5.create_dataset('vacuum', data=vacuum_5)
    dset_sqz_5 = dset_5.create_dataset('squeezing', data=squeezing_5)
    dset_asqz_5 = dset_5.create_dataset('antisqueezing', data=antisqueezing_5)

    #Add metadata
    for key, val in meta_elec.items():
        dset_spec.attrs[key] = val
    for key, val in meta_elec_5.items():
        dset_5.attrs[key] = val

In [72]:
file = h5py.File('squeezing_data.hdf5', 'r')
list(file.keys())

['5MHz', 'spectrum']

In [73]:
list(file['5MHz'].items())

[('antisqueezing',
  <HDF5 dataset "antisqueezing": shape (10, 5000), type "<f8">),
 ('electronic', <HDF5 dataset "electronic": shape (5000,), type "<f8">),
 ('power', <HDF5 dataset "power": shape (10,), type "<f8">),
 ('squeezing', <HDF5 dataset "squeezing": shape (10, 5000), type "<f8">),
 ('times', <HDF5 dataset "times": shape (5000,), type "<f8">),
 ('vacuum', <HDF5 dataset "vacuum": shape (5000,), type "<f8">)]

In [60]:
file['5MHz']['vacuum']

<HDF5 dataset "vacuum": shape (5000,), type "<f8">